In [2]:
#Adicionando as bibliotecas necessárias para as análises e soluções do Dataset

from gurobipy import Model, GRB
import pandas as pd
import os 
import tempfile
import subprocess
import folium
import webbrowser
import requests
import json

In [3]:
#Lista das Fábricas e Clientes 

origens =["vitoria","campinas","belo horizonte"]
destinos = ["rio de janeiro","volta redonda","ipatinga","ribeirao preto","governador valadares"]

print(origens)
print(destinos)


['vitoria', 'campinas', 'belo horizonte']
['rio de janeiro', 'volta redonda', 'ipatinga', 'ribeirao preto', 'governador valadares']


In [4]:
#Definindo a Quantidade de Ofertas de Veículos por Origem e as Demandas nos clientes

ofertas = {"vitoria":40, "campinas":35, "belo horizonte":25}
demandas = {"rio de janeiro":30, "volta redonda":12, "ipatinga":16, "ribeirao preto":24, "governador valadares":18}


### Abaixo os custos será representado pela distância que temos em KM das cidades.

In [5]:
#Criando os custos que teremos de Transporte de cada origem para cada destino

custos = {
    ("vitoria", "rio de janeiro"):517, ("vitoria", "volta redonda"): 555, ("vitoria", "ipatinga"): 405, ("vitoria", "ribeirao preto"): 1023, ("vitoria", "governador valadares"): 360,
    ("campinas", "rio de janeiro"): 493, ("campinas", "volta redonda"): 375, ("campinas", "ipatinga"): 790, ("campinas","ribeirao preto"): 223, ("campinas","governador valadares"): 890,
    ("belo horizonte", "rio de janeiro"): 441, ("belo horizonte", "volta redonda"): 439, ("belo horizonte", "ipatinga"): 212, ("belo horizonte", "ribeirao preto"): 513, ("belo horizonte", "governador valadares"): 315,
}

### Criando as Variáveis de Descisão e a Função Objetivo

In [6]:
modelo = Model("problema_de_transporte")

Set parameter Username


In [11]:
# Variáveis de decisão (quantidade a ser transportada de cada origem para cada destino)

var_decisao = {}
for origem in origens:
    for destino in destinos:
        var_decisao[origem, destino] = modelo.addVar(vtype=GRB.CONTINUOUS, name=f"Quantidade_{origem}_{destino}")
        
# Adicione a função objetivo (minimizar os custos de transporte)
modelo.setObjective(
    sum(var_decisao[origem, destino] * custos[origem, destino] for origem in origens for destino in destinos),
    GRB.MINIMIZE
)

In [12]:
# restrições de oferta e demanda

for origem in origens:
    modelo.addConstr(sum(var_decisao[origem, destino] for destino in destinos) <= ofertas[origem], f"Oferta_{origem}")

for destino in destinos:
    modelo.addConstr(sum(var_decisao[origem, destino] for origem in origens) >= demandas[destino], f"Demanda_{destino}")

### Resolução do Problema

In [14]:
# Rodar o modelo de Otimização

modelo.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Pentium(R) CPU 5405U @ 2.30GHz, instruction set [SSE2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 8 rows, 45 columns and 30 nonzeros
Model fingerprint: 0x7b6cd5c6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+01]
Presolve removed 0 rows and 30 columns
Presolve time: 0.04s
Presolved: 8 rows, 15 columns, 30 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.500000e+01   0.000000e+00      0s
       9    3.4690000e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.08 seconds (0.00 work units)
Optimal objective  3.469000000e+04


### Imprimindo a Solução ÓTIMA

In [246]:
# Imprimir a solução

print("Status:", modelo.status)
print("Solução ótima:")
for origem in origens:
    for destino in destinos:
        quantidade = var_decisao[origem, destino].x
        if quantidade > 0:
            print(f"Enviar {quantidade} veículos de {origem} para {destino}")

Status: 2
Solução ótima:
Enviar 22.0 veículos de vitoria para rio de janeiro
Enviar 18.0 veículos de vitoria para governador valadares
Enviar 11.0 veículos de campinas para volta redonda
Enviar 24.0 veículos de campinas para ribeirao preto
Enviar 8.0 veículos de belo horizonte para rio de janeiro
Enviar 1.0 veículos de belo horizonte para volta redonda
Enviar 16.0 veículos de belo horizonte para ipatinga


### Iremos criar um Mapa exemplificando como ficará a Distribuição

In [15]:
# Coordenadas aproximadas do Brasil

mapa = folium.Map(location=[-22.9068, -43.1729], zoom_start=5)

# Adicionando marcadores das Latitudes e Longitudes das Fábricas e Clientes

fabricas = {"vitoria": {"latitude": -20.3155, "longitude": -40.3128},
            "campinas": {"latitude": -22.9071, "longitude": -47.0633},
            "belo horizonte": {"latitude": -19.8157, "longitude": -43.9542}}

clientes = {"rio de janeiro": {"latitude": -22.9068, "longitude": -43.1729},
            "volta redonda": {"latitude": -22.5207, "longitude": -44.1026},
            "ipatinga": {"latitude": -19.4687, "longitude": -42.5367},
            "ribeirao preto": {"latitude": -21.1704, "longitude": -47.8103},
            "governador valadares": {"latitude": -18.8499, "longitude": -41.9491}}

for origem in origens:
    folium.Marker(location=[fabricas[origem]["latitude"], fabricas[origem]["longitude"]],
                  icon=folium.Icon(color='blue', icon='industry', prefix='fa')).add_to(mapa)

for destino in destinos:
    folium.Marker(location=[clientes[destino]["latitude"], clientes[destino]["longitude"]],
                  icon=folium.Icon(color='red', icon='warehouse', prefix='fa')).add_to(mapa)

# Adicionar linhas para mostrar a rota

for origem in origens:
    for destino in destinos:
        quantidade = var_decisao[origem, destino].x
        if quantidade > 0:
            folium.PolyLine(locations=[
                [fabricas[origem]["latitude"], fabricas[origem]["longitude"]],
                [clientes[destino]["latitude"], clientes[destino]["longitude"]]
            ], color='blue', weight=2).add_to(mapa)

# Salvar o mapa como um arquivo HTML
caminho_arquivo_html = os.path.join(tempfile.gettempdir(), "mapa_distribuicao.html")
mapa.save(caminho_arquivo_html)

# Abrir o arquivo HTML no navegador padrão
webbrowser.open(caminho_arquivo_html)

# Imprimir mensagem de conclusão
print(f"Mapa de distribuição salvo e aberto em: {caminho_arquivo_html}")

Mapa de distribuição salvo e aberto em: C:\Users\Gabriel Souza\AppData\Local\Temp\mapa_distribuicao.html
